# eBook to Audio project

### install libs

In [5]:
pip install gTTS pydub ebooklib bs4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)

   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [bs4]

Note: you may need to restart the kernel to use updated packages.


## extract chapters

Chapter 702: Moving on



When leaving the Parasites to their own devices, there was the chance that they could use this time to run away. To escape the planet, however, where would they even go? Who would they run off to? Wevil, had promised Quinn that he would follow him from now onwards, and to leave everything to him and Hana.Even though Tony was the vice leader, the faction members actually had the most respect for these two. Whenever something had gone wrong, they would be the ones that tried to reason with Mantis, to give them part of the antidote earlier than needed. While Tony would be the one who was Mantis's yes men.Saying Yes to every suggestion and everything he did."Alright everyone, it's time for you all to decide!" Wevil shouted from on top of the platform. The faction members were recovering and they had gained enough strength to move again. But they were unaware of the bombshell that Wevil was about to drop on them all. "You have a choice…."*****Heading back, Quinn an

In [ ]:
import re
from ebooklib import epub, ITEM_DOCUMENT
from bs4 import BeautifulSoup

def clean_text_for_tts(text):
    # Ensure spacing after punctuation
    text = re.sub(r'([.?!])(?=\w)', r'\1 ', text)

    # Add longer pause for sentence ends
    text = text.replace('. ', '. ... ')
    text = text.replace('? ', '? ... ')
    text = text.replace('! ', '! ... ')

    # Fix smashed dialogue punctuation
    text = re.sub(r'(\.|\!|\?)(")', r'\1 \2', text)

    # Add pause after newlines (e.g., between paragraphs)
    text = re.sub(r'\n+', '\n... \n', text)

    return text.strip()

def extract_actual_chapters(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []

    for idx, item in enumerate(book.get_items(), start=1):
        if item.get_type() != ITEM_DOCUMENT:
            continue

        soup = BeautifulSoup(item.get_content(), 'html.parser')
        text = soup.get_text().strip()

        if idx == 1:
            continue  # Skip full dump
        if idx == 2 and "Chapter 701" in text:
            parts = re.split(r"(Chapter\s+\d+[:\-]?\s+)", text)
            for i in range(1, len(parts) - 1, 2):
                title = parts[i].strip()
                body = parts[i + 1].strip()
                if len(body) > 30:
                    full = f"{title}\n\n{body}"
                    cleaned = clean_text_for_tts(full)
                    chapters.append(cleaned)
            continue

        match = re.search(r"(Chapter\s+\d+[:\-]?\s+.*)", text, re.IGNORECASE)
        if match:
            chapter_start = match.start()
            chapter_text = text[chapter_start:].strip()
            if len(chapter_text) > 50:
                cleaned = clean_text_for_tts(chapter_text)
                chapters.append(cleaned)

    return chapters


Chapter 702: Moving on



When leaving the Parasites to their own devices, there was the chance that they could use this time to run away. To escape the planet, however, where would they even go? Who would they run off to? Wevil, had promised Quinn that he would follow him from now onwards, and to leave everything to him and Hana.Even though Tony was the vice leader, the faction members actually had the most respect for these two. Whenever something had gone wrong, they would be the ones that tried to reason with Mantis, to give them part of the antidote earlier than needed. While Tony would be the one who was Mantis's yes men.Saying Yes to every suggestion and everything he did."Alright everyone, it's time for you all to decide!" Wevil shouted from on top of the platform. The faction members were recovering and they had gained enough strength to move again. But they were unaware of the bombshell that Wevil was about to drop on them all. "You have a choice…."*****Heading back, Quinn an

In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import os

def save_chapters_to_m4a(chapters, output_dir="chapters_m4a", max_chapters=10, bitrate="64k"):
    os.makedirs(output_dir, exist_ok=True)

    for i, chapter in enumerate(chapters[:max_chapters], start=1):
        print(f"🔊 Converting Chapter {i} to M4A...")
        try:
            # Step 1: Generate temp MP3
            temp_mp3 = f"temp_chapter_{i}.mp3"
            tts = gTTS(text=chapter, lang='en')
            tts.save(temp_mp3)

            # Step 2: Convert to M4A (AAC format)
            audio = AudioSegment.from_mp3(temp_mp3)
            m4a_path = os.path.join(output_dir, f"chapter_{i}.m4a")
            audio.export(m4a_path, format="ipod", bitrate=bitrate)

            os.remove(temp_mp3)  # Clean up
            print(f"✅ Saved: {m4a_path}")

        except Exception as e:
            print(f"❌ Error in Chapter {i}: {e}")

In [ ]:
chapters = extract_actual_chapters("audio_books/9kafe.com-my-vampire-system-c701-1400.epub")
print(chapters[2])  # Should print Chapter 703 with body

save_chapters_to_m4a(chapters, max_chapters=1)